In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

########################################################
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


########################################################

2.1.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9771148416486218238
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10068632535
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5198687023615967800
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Default GPU Device: /device:GPU:0


In [ ]:
from tika import parser
filepath = "input/babits_összes.pdf"

raw = parser.from_file(filepath)

In [5]:
print(raw['content'])




































In Horatium  

Gyülöllek: távol légy, alacsony tömeg!
ne rezzents nyelvet: hadd dalolok soha
   nem hallott verseket ma, múzsák
      papja, erős fiatal füleknek.
Nézz fel az égre: barna cigány ködök -
nézz szét a vízen: fürge fehér habok
   örök cseréjükért hálásak
      halld, Aiolost hogyan áldják, dallal.
A láng is hullám. Szüntelenül lobog
főnix-világunk. Igy nem is él soha,
   mi soha meg nem halt. Halálnak
      köszönöd életedet, fü és vad!
Minden e földön, minden a föld fölött
folytonfolyású, mint csobogó patak
   s »nem lépsz be kétszer egy patakba«,
      így akarák Thanatos s Aiolos.
Ekként a dal is légyen örökkön új,
a régi eszme váltson ezer köpenyt,
   s a régi forma új eszmének
      öltönyeként kerekedjen újra.
S ha Tibur gazdadalnoka egykor ily
mértéken zengte a megelégedést,
   hadd dalljam rajt ma himnuszát én
      a soha-meg-nem-elégedésnek!
Minden a földön, minden a föld fölött
folytonfolyású, mint hegyi záporár,
   hullámtörés,

In [8]:
raw_text = raw['content']
"".join(sorted(set(raw_text.lower())))

'\t\n !"#\'()*+,-./0123456789:;?[]abcdefghijklmnopqrstuvwxyz«»áâäèéëíîóôöúüőű'

In [15]:
filtered = [char for char in "()*+0123456789§«»–…✝'[]#\\"]


for char in filtered:
    raw_text = raw_text.replace(char, '')

raw_text = raw_text.replace('\n', ' ')
raw_text = raw_text.replace('\t', ' ')

"".join(sorted(set(raw_text.lower())))
print(raw_text[5480:5648])

izott polgárok, szűk utak.  Frankhon. Vidám, könnyelmü nép. Mennyi kirakat, mennyi kép! Mekkora nyüzsgés, mennyi hang: masina, csengő, kürt, harang.  Angolhon. Hidak és


In [12]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([raw_text])

In [13]:
tokenizer.texts_to_sequences(["First"])

[[29, 11, 10, 5, 4]]

In [14]:
tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(["First"]))

['f i r s t']

In [ ]:
# # Creating a reverse dictionary
# reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# # Function takes a tokenized sentence and returns the words
# def sequence_to_text(list_of_indices):
#     # Looking up words in dictionary
#     words = [reverse_word_map.get(letter) for letter in list_of_indices]
#     return(words)


# sequence_to_text([50, 10, 8, 7, 3])

In [16]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

max_id, dataset_size

(51, 1)

In [19]:
# Encoding the full text (substract 1 to get IDs from 0-38 rather than 1-39)
[encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) - 1
# train_size = dataset_size * 90 // 100
train_size = len(encoded) * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])


# dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# [encoded] = np.array(tokenizer.texts_to_sequences([raw_text])) -1

n_steps = 100
window_length = n_steps + 1 
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
# dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 50
dataset= dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:, 1:]))

dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [20]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(50, 100, 51) (50, 100)


In [21]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax')),
# ])


model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)






# model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer="adam")


Train for 12408 steps
Epoch 1/10
12408/12408 [==============================] - 151s 12ms/step - loss: 1.3814
Epoch 2/10
12408/12408 [==============================] - 161s 13ms/step - loss: 1.2640
Epoch 3/10
12408/12408 [==============================] - 163s 13ms/step - loss: 1.2820
Epoch 4/10
12408/12408 [==============================] - 163s 13ms/step - loss: 1.3071
Epoch 5/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.3244
Epoch 6/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.3461
Epoch 7/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.3839
Epoch 8/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.3833
Epoch 9/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.3910
Epoch 10/10
12408/12408 [==============================] - 162s 13ms/step - loss: 1.4042


In [ ]:
# history = model.fit(dataset, epochs = 20)

# %load_ext tensorboard
# %tensorboard --logdir=logs

In [22]:
model_name = "Babits_2_GRU_1_TimeDist_Dense_no_dropout"
# !mkdir -p model_name
model.save(model_name)

# new_model = tf.keras.models.load_model('saved_model/my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Babits_2_GRU_1_TimeDist_Dense_no_dropout\assets


In [23]:
# USING THE MODEL TO PRED CHAR
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["ej mi a k"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'é'

In [ ]:
# tf.random.set_seed(42)
# tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()


In [24]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [25]:
tf.random.set_seed(42)
next_char("valam", temperature=1)

'a'

In [26]:

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [30]:
tf.random.set_seed(42)

# print(complete_text("t", temperature=0.2))
print(complete_text("Magyar ",n_chars=250 ,temperature=0.8))
# print(complete_text("t", temperature=2))

Magyar életed zúzott éren künn örökre ragyog.  mint kapitus,  . izzad égerek? s mind nyila voltam: talán meg napás, sötétencsét, jajgat bucsályán. mint a semmi rossza elém, mely rossz ködék sötét vagy? óna s sereget xak szemem nem szakadt lettemet csontban 
